1. Create & Prepare embedding dataset

In [1]:
from datasets import load_dataset
 
# Load dataset from the hub
dataset = load_dataset('csv', data_files='../dataset/QA_data.csv', split="train")


# Shuffle trước để chọn ngẫu nhiên
dataset = dataset.shuffle(seed=42)

# Lấy 1/100 số dòng
# dataset = dataset.select(range(len(dataset) // 100))


print("Before:", len(dataset))
dataset = dataset.filter(lambda x: x["Answer"])
print("After:", len(dataset))
# rename columns
dataset = dataset.rename_column("Question", "anchor")
dataset = dataset.rename_column("Answer", "positive")
 
# Add an id column to the dataset
dataset = dataset.add_column("id", range(len(dataset)))
 
# split dataset into a 10% test set
dataset = dataset.train_test_split(test_size=0.1)
 
# save datasets to disk
dataset["train"].to_pandas().to_json("train_dataset.json", orient="records", lines=True, force_ascii=False)
dataset["test"].to_pandas().to_json("test_dataset.json", orient="records", lines=True, force_ascii=False)


d:\HocTap\ChatBot\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Before: 197379
After: 159501


2. Create baseline and evaluate pretrained model

In [2]:

from datasets import load_dataset, concatenate_datasets
 
# load test dataset
test_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")
corpus_dataset = concatenate_datasets([train_dataset, test_dataset])
 
# Convert the datasets to dictionaries
corpus = dict(
    zip(corpus_dataset["id"], corpus_dataset["positive"])
)  # Our corpus (cid => document)
queries = dict(
    zip(test_dataset["id"], test_dataset["anchor"])
)  # Our queries (qid => question)
 
# Create a mapping of relevant document (1 in our case) for each query
relevant_docs = {}  # Query ID to relevant documents (qid => set([relevant_cids])
for q_id in queries:
    relevant_docs[q_id] = [q_id]
 
 


Generating train split: 15951 examples [00:00, 361251.10 examples/s]
Generating train split: 143550 examples [00:00, 603611.63 examples/s]


In [3]:
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.evaluation import (
    InformationRetrievalEvaluator,
    SequentialEvaluator,
)
from sentence_transformers.util import cos_sim

model_id = "bkai-foundation-models/vietnamese-bi-encoder"  # Hugging Face model ID
matryoshka_dimensions = [768, 512, 256, 128, 64] # Important: large to small
 
# Load a model
model = SentenceTransformer(
    model_id, device="cuda" if torch.cuda.is_available() else "cpu"
)

matryoshka_evaluators = []
# Iterate over the different dimensions
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,  # Truncate the embeddings to a certain dimension
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)
 
# Create a sequential evaluator
evaluator = SequentialEvaluator(matryoshka_evaluators)

In [4]:
# Evaluate the model
results = evaluator(model)
for k,v in results.items():
    print(k, v)

print("=======================")
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print
    print(f"{key}: {results[key]}")

dim_768_cosine_accuracy@1 0.23227383863080683
dim_768_cosine_accuracy@3 0.35847282302050026
dim_768_cosine_accuracy@5 0.4165256096796439
dim_768_cosine_accuracy@10 0.4767726161369193
dim_768_cosine_precision@1 0.23227383863080683
dim_768_cosine_precision@3 0.11949094100683343
dim_768_cosine_precision@5 0.08330512193592877
dim_768_cosine_precision@10 0.04767726161369193
dim_768_cosine_recall@1 0.23227383863080683
dim_768_cosine_recall@3 0.35847282302050026
dim_768_cosine_recall@5 0.4165256096796439
dim_768_cosine_recall@10 0.4767726161369193
dim_768_cosine_ndcg@10 0.34935886247371617
dim_768_cosine_mrr@10 0.309086856275119
dim_768_cosine_map@100 0.31452434828195525
dim_512_cosine_accuracy@1 0.23108269074039245
dim_512_cosine_accuracy@3 0.35571437527427746
dim_512_cosine_accuracy@5 0.4132656259795624
dim_512_cosine_accuracy@10 0.4730110964829791
dim_512_cosine_precision@1 0.23108269074039245
dim_512_cosine_precision@3 0.11857145842475914
dim_512_cosine_precision@5 0.08265312519591249
dim

3. Define loss function with Matryoshka Representation

In [5]:
from sentence_transformers import SentenceTransformerModelCardData, SentenceTransformer
 
# Hugging Face model ID: https://huggingface.co/BAAI/bge-base-en-v1.5
model_id = "bkai-foundation-models/vietnamese-bi-encoder"
 
# load model with SDPA for using Flash Attention 2
model = SentenceTransformer(
    model_id,
    model_kwargs={"attn_implementation": "sdpa"},
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="BGE base Financial Matryoshka",
    ),
)
train_dataset = load_dataset("json", data_files="train_dataset.json", split="train")
train_dataset

Dataset({
    features: ['anchor', 'Context', 'positive', 'Answer_Start', 'Answer_End', 'id'],
    num_rows: 143550
})

In [6]:
train_dataset[0]

{'anchor': 'Tôi bị nhiễm covid, đã hoàn thành điều trị, về cách ly tại nhà dc 4 ngày, giờ ko ho ko sốt, nhưng cổ có mắc đàm, vòm họng đỏ, hơi rát, vì có bị viêm amidan mãn tính, vậy cho e hỏi e có thể uống thuốc gì dc ạ, e còn thuốc điều trị amidan có uống dc ko',
 'Context': 'Chào chị Dạ chào bác sỹ Chị đặt giúp em một lịch tư vấn online miễn phí. Em gọi điện hỏi thêm triệu chứng và tư vấn cụ thể cho chị nhé Chị ấn vào ảnh đại diện của em, ấn tư vấn trực tuyến, ấn xác nhận và đặt lịch ạ Cảm ơn chị',
 'positive': 'Chào chị Dạ chào bác sỹ Chị đặt giúp em một lịch tư vấn online miễn phí. Em gọi điện hỏi thêm triệu chứng và tư vấn cụ thể cho chị nhé Chị ấn vào ảnh đại diện của em, ấn tư vấn trực tuyến, ấn xác nhận và đặt lịch ạ Cảm ơn chị',
 'Answer_Start': 0,
 'Answer_End': 225,
 'id': 88434}

In [7]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss
 
matryoshka_dimensions = [768, 512, 256, 128, 64]  # Important: large to small
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

4. Fine-tune embedding model with SentenceTransformersTrainer

In [8]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
 
# load train dataset again
 
# define training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="sample", # output directory and hugging face model ID
    num_train_epochs=10,                         # number of epochs
    per_device_train_batch_size=16,             # train batch size
    gradient_accumulation_steps=8,             # for a global batch size of 512
    per_device_eval_batch_size=8,    
    #gradient_checkpointing=True,
    warmup_ratio=0.1,                           # warmup ratio
    learning_rate=2e-5,                         # learning rate, 2e-5 is a good value
    lr_scheduler_type="cosine",                 # use constant learning rate scheduler
    optim="adamw_torch_fused",                  # use fused adamw optimizer
    # tf32=True,                                  # use tf32 precision
    bf16=True,                                  # use bf16 precision
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="epoch",                      # evaluate after each epoch
    save_strategy="epoch",  
    # save_steps = 500,
    logging_steps=10,                           # log every 10 steps
    save_total_limit=3,                         # save only the last 3 models
    load_best_model_at_end=True,                # load the best model when training ends
    metric_for_best_model="eval_dim_128_cosine_ndcg@10",  # Optimizing for the best ndcg@10 score for the 128 dimension
    report_to = "none",
)

In [9]:
from sentence_transformers import SentenceTransformerTrainer
 
trainer = SentenceTransformerTrainer(
    model=model, # bg-base-en-v1
    args=args,  # training arguments
    train_dataset=train_dataset.select_columns(
        ["anchor", "positive"]
    ),  # training dataset
    loss=train_loss,
    evaluator=evaluator,
)

In [10]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()
 
# save the best model
trainer.save_model()
 
# # push model to hub
# trainer.model.push_to_hub("bge-base-financial-matryoshka")

Epoch,Training Loss,Validation Loss,Dim 768 Cosine Accuracy@1,Dim 768 Cosine Accuracy@3,Dim 768 Cosine Accuracy@5,Dim 768 Cosine Accuracy@10,Dim 768 Cosine Precision@1,Dim 768 Cosine Precision@3,Dim 768 Cosine Precision@5,Dim 768 Cosine Precision@10,Dim 768 Cosine Recall@1,Dim 768 Cosine Recall@3,Dim 768 Cosine Recall@5,Dim 768 Cosine Recall@10,Dim 768 Cosine Ndcg@10,Dim 768 Cosine Mrr@10,Dim 768 Cosine Map@100,Dim 512 Cosine Accuracy@1,Dim 512 Cosine Accuracy@3,Dim 512 Cosine Accuracy@5,Dim 512 Cosine Accuracy@10,Dim 512 Cosine Precision@1,Dim 512 Cosine Precision@3,Dim 512 Cosine Precision@5,Dim 512 Cosine Precision@10,Dim 512 Cosine Recall@1,Dim 512 Cosine Recall@3,Dim 512 Cosine Recall@5,Dim 512 Cosine Recall@10,Dim 512 Cosine Ndcg@10,Dim 512 Cosine Mrr@10,Dim 512 Cosine Map@100,Dim 256 Cosine Accuracy@1,Dim 256 Cosine Accuracy@3,Dim 256 Cosine Accuracy@5,Dim 256 Cosine Accuracy@10,Dim 256 Cosine Precision@1,Dim 256 Cosine Precision@3,Dim 256 Cosine Precision@5,Dim 256 Cosine Precision@10,Dim 256 Cosine Recall@1,Dim 256 Cosine Recall@3,Dim 256 Cosine Recall@5,Dim 256 Cosine Recall@10,Dim 256 Cosine Ndcg@10,Dim 256 Cosine Mrr@10,Dim 256 Cosine Map@100,Dim 128 Cosine Accuracy@1,Dim 128 Cosine Accuracy@3,Dim 128 Cosine Accuracy@5,Dim 128 Cosine Accuracy@10,Dim 128 Cosine Precision@1,Dim 128 Cosine Precision@3,Dim 128 Cosine Precision@5,Dim 128 Cosine Precision@10,Dim 128 Cosine Recall@1,Dim 128 Cosine Recall@3,Dim 128 Cosine Recall@5,Dim 128 Cosine Recall@10,Dim 128 Cosine Ndcg@10,Dim 128 Cosine Mrr@10,Dim 128 Cosine Map@100,Dim 64 Cosine Accuracy@1,Dim 64 Cosine Accuracy@3,Dim 64 Cosine Accuracy@5,Dim 64 Cosine Accuracy@10,Dim 64 Cosine Precision@1,Dim 64 Cosine Precision@3,Dim 64 Cosine Precision@5,Dim 64 Cosine Precision@10,Dim 64 Cosine Recall@1,Dim 64 Cosine Recall@3,Dim 64 Cosine Recall@5,Dim 64 Cosine Recall@10,Dim 64 Cosine Ndcg@10,Dim 64 Cosine Mrr@10,Dim 64 Cosine Map@100,Sequential Score
1,4.883900,No log,0.372265,0.525547,0.571375,0.620463,0.372265,0.175182,0.114275,0.062046,0.372265,0.525547,0.571375,0.620463,0.496825,0.457113,0.462092,0.368629,0.523729,0.571438,0.618645,0.368629,0.174576,0.114288,0.061864,0.368629,0.523729,0.571438,0.618645,0.494357,0.454411,0.459396,0.363676,0.516958,0.565043,0.613817,0.363676,0.172319,0.113009,0.061382,0.363676,0.516958,0.565043,0.613817,0.489135,0.449090,0.454043,0.351890,0.503354,0.552567,0.603536,0.351890,0.167785,0.110513,0.060354,0.351890,0.503354,0.552567,0.603536,0.477686,0.437332,0.442267,0.324933,0.471569,0.519529,0.571626,0.324933,0.157190,0.103906,0.057163,0.324933,0.471569,0.519529,0.571626,0.447395,0.407659,0.412994,0.447395
2,4.087900,No log,0.388878,0.540593,0.587926,0.637390,0.388878,0.180198,0.117585,0.063739,0.388878,0.540593,0.587926,0.637390,0.513486,0.473720,0.478893,0.387938,0.536518,0.586609,0.634694,0.387938,0.178839,0.117322,0.063469,0.387938,0.536518,0.586609,0.634694,0.511661,0.472181,0.477426,0.382421,0.534136,0.579337,0.627359,0.382421,0.178045,0.115867,0.062736,0.382421,0.534136,0.579337,0.627359,0.505731,0.466632,0.472020,0.371325,0.521723,0.569243,0.617203,0.371325,0.173908,0.113849,0.061720,0.371325,0.521723,0.569243,0.617203,0.494705,0.455363,0.460718,0.346123,0.492822,0.539966,0.590935,0.346123,0.164274,0.107993,0.059093,0.346123,0.492822,0.539966,0.590935,0.468365,0.429123,0.434847,0.468365
3,3.371800,No log,0.398157,0.547991,0.594634,0.642468,0.398157,0.182664,0.118927,0.064247,0.398157,0.547991,0.594634,0.642468,0.521257,0.482317,0.487589,0.395148,0.546047,0.591938,0.640900,0.395148,0.182016,0.118388,0.064090,0.395148,0.546047,0.591938,0.640900,0.519017,0.479852,0.485138,0.391574,0.541283,0.586860,0.634945,0.391574,0.180428,0.117372,0.063494,0.391574,0.541283,0.586860,0.634945,0.514033,0.475204,0.480522,0.379412,0.532130,0.576704,0.626105,0.379412,0.177377,0.115341,0.062610,0.379412,0.532130,0.576704,0.626105,0.503916,0.464651,0.469884,0.363175,0.507429,0.554385,0.601655,0.363175,0.169143,0.110877,0.060166,0.363175,0.507429,0.554385,0.6016

5. Evaluate fine-tuned model against baseline

In [11]:
from sentence_transformers import SentenceTransformer
 
fine_tuned_model = SentenceTransformer(
    args.output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)
# Evaluate the model
results = evaluator(fine_tuned_model)
 
# # COMMENT IN for full results
for k,v in results.items():
    print(k, v)
    
print("=======================")
 
# Print the main score
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print(f"{key}: {results[key]}")

dim_768_cosine_accuracy@1 0.4199109773681901
dim_768_cosine_accuracy@3 0.5694313836123127
dim_768_cosine_accuracy@5 0.6146323114538274
dim_768_cosine_accuracy@10 0.6627797630242618
dim_768_cosine_precision@1 0.4199109773681901
dim_768_cosine_precision@3 0.18981046120410425
dim_768_cosine_precision@5 0.12292646229076545
dim_768_cosine_precision@10 0.06627797630242617
dim_768_cosine_recall@1 0.4199109773681901
dim_768_cosine_recall@3 0.5694313836123127
dim_768_cosine_recall@5 0.6146323114538274
dim_768_cosine_recall@10 0.6627797630242618
dim_768_cosine_ndcg@10 0.5422040066911552
dim_768_cosine_mrr@10 0.5034606229593954
dim_768_cosine_map@100 0.5087533694160415
dim_512_cosine_accuracy@1 0.418218293523917
dim_512_cosine_accuracy@3 0.5677386997680396
dim_512_cosine_accuracy@5 0.6136292395461099
dim_512_cosine_accuracy@10 0.6612751551626858
dim_512_cosine_precision@1 0.418218293523917
dim_512_cosine_precision@3 0.18924623325601322
dim_512_cosine_precision@5 0.12272584790922199
dim_512_cosine